In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz

# Part 1: Make Decision Tree for Binary Input Values

In [2]:
# make a dataframe using binary input variables
# A, B, C are binary input features
# Y is the binary target label

data = {'A':[0,1,0,1,0,1],
        'B':[1,1,0,1,1,0],
        'C':[1,1,0,0,0,1],
        'Y':[0,0,0,1,1,1]}
data = pd.DataFrame(data)
data

,A,B,C,Y
0,0,1,1,0
1,1,1,1,0
2,0,0,0,0
3,1,1,0,1
4,0,1,0,1
5,1,0,1,1


In [3]:
X = data.drop(columns = ['Y'])
Y = data[['Y']]

In [4]:
X

,A,B,C
0,0,1,1
1,1,1,1
2,0,0,0
3,1,1,0
4,0,1,0
5,1,0,1


In [5]:
Y

,Y
0,0
1,0
2,0
3,1
4,1
5,1


# SKLearn Decision Tree

In [6]:
clf = DecisionTreeClassifier(criterion = "entropy", max_depth = 2)
clf.fit(X, Y)
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("dt") 

'dt.pdf'

# DIY Decision Tree with Information Gain method
- Entropy $H(X) = -\sum_i p_i \log_2 p_i$
- $p_i$ is the probability of a class within that node of a decision tree
- Information gain = Entropy of parent - Average entropy of children
- Choose split with most information gain

In [14]:
class MyDecisionTree:
    def __init__(self, max_depth = 2, num_bins = 10):
        self.max_depth = max_depth
        self.num_bins = num_bins
        self.split = []
        
    ''' Fits the decision tree based on the initial inputs X and outputs Y '''
    def fit(self, X, Y):
        # make this a dataframe if not already one
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        if not isinstance(Y, pd.DataFrame):
            Y = pd.DataFrame(Y)
        
        self.default_class = Y.mode().values[0]
        self.split = self.tree_recurse(X, Y, 0)
        
    ''' This function splits based on the best split by information gain method '''
    def tree_recurse(self, X, Y, depth):
        # if the node has no samples, just return default class 0
        if len(Y) == 0:
            return self.default_class
        
        # if the node is pure
        # return majority class
        if len(Y.value_counts()) == 1:
            return Y.mode().values[0]
        
        # do not want to oversplit beyond the max_depth specified
        # return majority class
        if depth >= self.max_depth:
            return Y.mode().values[0]
        
        variables = list(X.columns)
        best_entropy = 10000
        best_split = ''
        best_value = None
        best_x_A, best_x_B, best_y_A, best_y_B = None, None, None, None
        
        # find out the best variable to split on, and its value
        for var in variables:
            # split into num_bins number of bins
            for value in np.linspace(X[var].min(), X[var].max(), self.num_bins):
                child_A = X[var] <= value
                child_B = X[var] > value
                x_A = X[child_A]
                x_B = X[child_B]
                y_A = Y[child_A]
                y_B = Y[child_B]
                
                # find out entropy of the children
                average_entropy = len(y_A)/len(Y)*self.entropy(y_A) + len(y_B)/len(Y)*self.entropy(y_B)
                
                # if this is the best, add to the best split
                if average_entropy <= best_entropy:
                    best_entropy = average_entropy
                    best_split = var
                    best_value = value
                    best_x_A, best_x_B, best_y_A, best_y_B = pd.DataFrame(x_A), pd.DataFrame(x_B), pd.DataFrame(y_A), pd.DataFrame(y_B)
                    
        # recursively split trees based on the child nodes
        # return in the format [(best_split, best_value, best_entropy), outcome for child A, outcome for child B]
        return [(best_split, best_value, best_entropy), self.tree_recurse(best_x_A, best_y_A, depth+1), self.tree_recurse(best_x_B, best_y_B, depth+1)]
        
    ''' Predict the label for unseen data 
        Inputs: features X
        Output: predicted class/label Y
    '''
    def predict(self, X):
        # make this a dataframe if not already one
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        return np.array([self.recurse_predict(X.iloc[i,:], self.split) for i in range(X.shape[0])]).flatten()
        
    ''' Recursively do the prediction 
        X: input features
        split: information at the current branch '''
    def recurse_predict(self, X, split):
        # if split is not a tuple and not a list, then it is the desired class, return it
        if type(split) != tuple and type(split) != list:
            return split
        split_info, b1, b2 = split
        var, value, entropy = split_info
        next_split = b1 if X[var] <= value else b2
        return self.recurse_predict(X, next_split)
    
    ''' Calculates entropy based on label Y distribution '''
    def entropy(self, Y):
        count = np.array([x for x in Y.value_counts()])
        count = count/sum(count)
        return - sum([p_i*np.log2(p_i) if p_i > 0 else 0 for p_i in count])

In [15]:
myclf = MyDecisionTree(max_depth = 2)
myclf.fit(X, Y)
# check out the optimal split
myclf.split

[('C', 0.8888888888888888, 0.9182958340544896),
 [('B', 0.8888888888888888, -0.0), array([0]), array([1])],
 [('B', 0.8888888888888888, -0.0), array([1]), array([0])]]

In [16]:
# predict the output
myclf.predict(X)

array([0, 0, 0, 1, 1, 1])

# Looks like this is working
- Let us try this on the Wisconscin Breast Cancer Dataset

In [17]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

data = datasets.load_breast_cancer()
X, Y = data.data, data.target

In [19]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [20]:
pd.DataFrame(Y)

,0
0,0
1,0
2,0
3,0
4,0
...,...
564,0
565,0
566,0
567,0


# Let's run it on sklearn decision tree

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=42)

In [22]:
clf = DecisionTreeClassifier(criterion = "entropy", max_depth = 2)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("dt_wdbc") 

'dt_wdbc.pdf'

In [23]:
print('Accuracy', accuracy_score(Y_test, Y_pred))
print('Precision', precision_score(Y_test, Y_pred))
print('Recall', recall_score(Y_test, Y_pred))

Accuracy 0.9473684210526315
Precision 0.9623655913978495
Recall 0.9572192513368984


# Let's compare it to ours

In [27]:
myclf = MyDecisionTree(max_depth = 2, num_bins = 30)
myclf.fit(X_train, Y_train)
# check out the optimal split
myclf.split

[(27, 0.14255172413793102, 0.3910471527829722),
 [(23, 938.3241379310346, 0.30752903997168746), array([1]), array([0])],
 [(16, 0.1390813793103448, 0.08252505911074368), array([0]), array([1])]]

In [28]:
# predict the output
Y_pred = myclf.predict(X_test)

In [29]:
print('Accuracy', accuracy_score(Y_test, Y_pred))
print('Precision', precision_score(Y_test, Y_pred))
print('Recall', recall_score(Y_test, Y_pred))

Accuracy 0.9508771929824561
Precision 0.9675675675675676
Recall 0.9572192513368984


# Verdict: Seems pretty similar
- Note: The sklearn method does not seem to be doing a greedy split - they evaluate the best possible split